In [1]:
import pandas as pd
import re
import gensim
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize
from gensim.models.ldamodel import LdaModel,CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

C:\Users\samarth152721\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df2 = pd.read_csv('C:/Users/samarth152721/Downloads/complaints.csv')
df = df2[['Product','Consumer complaint narrative']]
df.dropna(inplace = True)
df.reset_index(drop = True,inplace = True)
df.head()

,Product,Consumer complaint narrative
0,Debt collection,transworld systems inc. \nis trying to collect...
1,"Credit reporting, credit repair services, or o...",I would like to request the suppression of the...
2,Debt collection,"Over the past 2 weeks, I have been receiving e..."
3,"Credit reporting, credit repair services, or o...",someone used my personal information to get me...
4,"Money transfer, virtual currency, or money ser...","I was sold access to an event digitally, of wh..."


In [3]:
def renaming(text):
    if text =='Prepaid card':
        return 'Credit card or prepaid card'
    elif text == 'Payday loan':
        return 'Loans'
    elif text == 'Credit reporting':
        return 'Credit reporting, credit repair services, or other personal consumer reports'
    elif text == 'Money transfers':
        return 'Money transfer, virtual currency, or money service'
    elif text == 'Credit card':
        return 'Credit card or prepaid card'
    elif text == 'Consumer Loan':
        return 'Loans'
    elif text == 'Vehicle loan or lease':
        return 'Loans'
    elif text == 'Payday loan, title loan, or personal loan':
        return 'Loans'
    elif text == 'Student loan':
        return 'Loans'
    elif text == 'Virtual currency':
        return 'Money transfer, virtual currency, or money service'
    else:
        return text

In [4]:
df['Product'] = df['Product'].progress_apply(renaming)

100%|██████████████████████████████████████████████████████████████████████| 496719/496719 [00:00<00:00, 906218.72it/s]


In [5]:
df['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    178886
Debt collection                                                                 107741
Mortgage                                                                         61901
Credit card or prepaid card                                                      52931
Loans                                                                            51189
Checking or savings account                                                      19418
Bank account or service                                                          14885
Money transfer, virtual currency, or money service                                9476
Other financial service                                                            292
Name: Product, dtype: int64

In [6]:
df2 = df[df['Product']=='Bank account or service']

In [7]:
df2.head()

,Product,Consumer complaint narrative
18610,Bank account or service,I applied for Bank of the west checking accoun...
22439,Bank account or service,Near the end of XXXX 2016 I opened a Citigold ...
22763,Bank account or service,I own a XXXX wanted to transfer money from XXX...
23685,Bank account or service,My bank account was tied to my XXXX. My XXXX w...
24057,Bank account or service,"Dear Scottrade, As mentioned in my separate le..."


In [8]:
stop_words = list(set(stopwords.words('english')))+list(punctuation)
lem = WordNetLemmatizer()

In [9]:
def cleaning(text):
    text = re.sub('XXXX',' UNKNOWN ',text)
    text = re.sub('XX/XX/','',text)
    text = re.sub('UNKNOWN   UNKNOWN','UNKNOWN',text)
    text = re.sub('UNKNOWN','',text)
    text = re.sub('\n',' ',text)
    text = re.sub('  ',' ',text)
    text = text.lower()
    words = word_tokenize(text)
    words = [w for w in words if w not in stop_words]
    words = [w for w in words if len(w)>2]
    words = [lem.lemmatize(w) for w in words]
    return words

In [10]:
df2['clean_text'] = df2['Consumer complaint narrative'].progress_apply(cleaning)

100%|███████████████████████████████████████████████████████████████████████████| 14885/14885 [00:39<00:00, 373.19it/s]


In [11]:
clean_doc = list(df2['clean_text'].values)

In [12]:
dictionary = Dictionary(clean_doc)

In [13]:
corpus = [dictionary.doc2bow(doc) for doc in clean_doc]

In [ ]:
%%time
ldamodel = LdaModel(corpus=corpus,id2word=dictionary,num_topics=5,random_state=42,update_every=1,passes=50, chunksize=100)

In [ ]:
print(ldamodel.print_topics())

In [ ]:
df2.head(10)